# Tarjeta de datos 4

#### Hipótesis 4: Las provincias situadas en el sur de España (aquellas al sur de Madrid) tienden a utilizar menos dispositivos de aprovechamiento de energías renovables que las del norte.  

Las diferencias climáticas entre el norte y sur de España pueden ser una variable crucial a la hora de instalar dispositivos de energía renovable pero no la única. El clima influye en cuanto a la cantidad de energía que se puede producir pero también a la consumida.

Además pueden influir el ámbito socioeconómico, las provincias con mayor poder adquisitivo podrían gastarse más en energías renovables o las más pobres quieren ahorrarse en las facturas de la luz. Otra posible influencia pueden ser los tamaños familiares o la edad.

### Preparación del entorno

In [ ]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import os
import warnings
warnings.filterwarnings("ignore")

DATA_PATH = os.path.join("..", "..", "data/silver/")
GOLD_DATA_PATH = os.path.join("..", "..", "data/gold/")
PROFILING_PATH = os.path.join("..", "..", "profiling/")

### Carga de Datos

In [ ]:
dispositivos_df = pd.read_csv(DATA_PATH + 'dispositivos_renovable.csv', sep=';', encoding='latin')
dispositivos_df.info()

In [ ]:
renta_df = pd.read_csv(DATA_PATH + 'renta_media_hogar.csv', sep=';', encoding='latin')
renta_df.info()

In [ ]:
hogar_df = pd.read_csv(DATA_PATH + 'tipo_nucleo_familiar.csv', sep=';', encoding='latin')
hogar_df.info()

In [ ]:
edad_df = pd.read_csv(DATA_PATH + 'distribucion_edad.csv', sep=';', encoding='latin')
edad_df.info()

In [ ]:
produccion_df = pd.read_csv(DATA_PATH + 'produccion_lugar.csv', sep=';', encoding='latin')
produccion_df.info()

### Procesamiento

- El dataFrame de la renta queremos normalizarlo para tener una visión global y como con todos los datos los queremos agrupados por provincias.

In [ ]:
scaler = MinMaxScaler()
renta_df['Renta normalizada'] = scaler.fit_transform(renta_df[['Total']])
renta_df = renta_df.groupby('Provincias')['Renta normalizada'].mean().reset_index()

- Para los datos familiares vamos a juntar algunas columnas que no nos aportan nada separadas y posterior las agrupamos por provincias.

In [ ]:
hogar_df['Número de hijos'] = hogar_df['Número de hijos'].replace({
    '0 hijos conviviendo': '0',
    '1 hijo conviviendo': '1',
    '2 o más hijos conviviendo': '2',
    '0 hijos conviviendo menores de 25 años': '0',
    '1 hijo conviviendo menor de 25 años': '1',
    '2 hijos o más conviviendo menores de 25 años': '2'
})
hogar_df['Tipo de núcleo familiar'] = hogar_df['Tipo de núcleo familiar'].replace({
    'Pareja casada, con o sin hijos convivientes': '2',
    'Pareja no casada, con o sin hijos convivientes': '2',
    'Padre sólo con hijos convivientes': '1',
    'Madre sóla con hijos convivientes': '1'
})
hogar_df.rename(columns={'Tipo de núcleo familiar': 'Número de Padres'}, inplace=True)
hogar_df = hogar_df.groupby(['Provincias', 'Número de hijos', 'Número de Padres'])['Total'].sum().reset_index()
hogar_df['Total_Provincia'] = hogar_df.groupby('Provincias')['Total'].transform('sum')
hogar_df['Total'] = hogar_df['Total'] / hogar_df['Total_Provincia']
hogar_df.drop(columns=['Total_Provincia'], inplace=True)
hogar_df['Tipo de familia'] = 'Familia de ' + hogar_df['Número de Padres'] + ' padres y ' + hogar_df['Número de hijos'] + ' hijos'
hogar_df = hogar_df.pivot(index='Provincias', columns='Tipo de familia', values='Total')


- En cuanto a la edad queremos saber un dato más significativo por provincia y por eso buscamos la mediana para lo cual tenemos que cambiar los datos de String con un rango a números. Elegimos el número medio del rango.

In [ ]:
edad_df['Edad (grupos quinquenales)'] = edad_df['Edad (grupos quinquenales)'].replace({
    'De 0 a 4 años': 2,
    'De 5 a 9 años': 7,
    'De 10 a 14 años  ': 12,
    'De 15 a 19 años  ': 17,
    'De 20 a 24 años': 22,
    'De 25 a 29 años': 27,
    'De 30 a 34 años': 32,
    'De 35 a 39 años': 37,
    'De 40 a 44 años': 42,
    'De 45 a 49 años': 47,
    'De 50 a 54 años': 52,
    'De 55 a 59 años': 57,
    'De 60 a 64 años': 62,
    'De 65 a 69 años': 67,
    'De 70 a 74 años  ': 72,
    'De 75 a 79 años  ': 77,
    'De 80 a 84 años  ': 82,
    'De 85 a 89 años  ': 87,
    'De 90 a 94 años  ': 92,
    'De 95 a 99 años  ': 97,
    '100 y más años': 100
})
mediana_edad_df = edad_df.groupby('Provincias').apply(lambda x: x.loc[x['Total'].cumsum() >= x['Total'].sum() / 2, 'Edad (grupos quinquenales)'].iloc[0]).reset_index()
mediana_edad_df.columns = ['Provincias', 'Mediana edad']

- La producción tiene las provincias listadas diferentes así que las igualamos al resto. Tiene los valores divididos por meses y nos interesa el anual para no centrarnos en distintas estaciones del año por tanto hacemos la media. Y a su vez le faltan algunas provincias que las añadimos y vamos a tener valores 0.

In [ ]:
produccion_df['Provincia'] = produccion_df['Provincia'].replace({
    'Alicante': 'Alicante/Alacant',
    'Álava': 'Araba/Álava',
    'Guipúzcoa': 'Gipuzkoa',
    'Lérida': 'Lleida',
    'La Rioja': 'Rioja, La',
    'Gerona': 'Girona',
    'Vizcaya': 'Bizkaia',
    'Castellón': 'Castellón/Castelló',
    'Valencia': 'Valencia/València',
    ' Madrid': 'Madrid',
    'La Coruña': 'Coruña, A',
    'Orense': 'Ourense',
    'Región de Murcia': 'Murcia'
})

produccion_agrupada = produccion_df.groupby('Provincia')['Potencia kWh'].agg(['mean']).reset_index()
produccion_agrupada.columns = ['Provincia', 'Producción media']

- Los dispositivos se dividen por los hogares que tienen y los que no. A nosotros nos interesa saber cuantos los tienen asi que vamos a escoger estos.

In [ ]:
dispositivos_agrupados = dispositivos_df.groupby(['Provincias', 'Dispone de dispositivo de energía renovable'])['Total'].sum().unstack().fillna(0)
dispositivos_agrupados['Porcentaje con dispositivo'] = dispositivos_agrupados['Si'] / (dispositivos_agrupados['Si'] + dispositivos_agrupados['No']) * 100

- Con todos los datasets transformados solo nos queda combinarlos en uno que lo hacemos a continuación escogiendo las columnas que nos interesan de cada uno

In [ ]:
combined_df = pd.DataFrame({
    'Provincias': dispositivos_agrupados.index,
    'Porcentaje con dispositivo': dispositivos_agrupados['Porcentaje con dispositivo'],
    'Renta normalizada': renta_df.set_index('Provincias')['Renta normalizada'],
    'Mediana edad': mediana_edad_df.set_index('Provincias')['Mediana edad'],
    'Producción media': produccion_agrupada.set_index('Provincia')['Producción media']
}).reset_index(drop=True)
combined_df = pd.merge(combined_df, hogar_df, on='Provincias', how='left')

In [ ]:
provincias_sur = [
    'Albacete', 'Alicante/Alacant', 'Almería', 'Badajoz', 'Castellón/Castelló',
    'Ciudad Real', 'Cuenca', 'Cáceres', 'Cádiz', 'Córdoba', 'Granada',
    'Guadalajara', 'Huelva', 'Jaén', 'Murcia', 'Málaga', 'Sevilla', 'Toledo',
    'Valencia/València'
]

provincias_norte = [
    'Araba/Álava', 'Asturias', 'Barcelona', 'Bizkaia', 'Burgos', 'Cantabria',
    'Coruña, A', 'Gipuzkoa', 'Girona', 'Huesca', 'León', 'Lleida', 'Lugo',
    'Navarra', 'Ourense', 'Palencia', 'Pontevedra', 'Rioja, La', 'Salamanca',
    'Segovia', 'Soria', 'Tarragona', 'Teruel', 'Valladolid', 'Zamora',
    'Zaragoza', 'Ávila'
]

df_sur = combined_df[combined_df['Provincias'].isin(provincias_sur)]
df_norte = combined_df[combined_df['Provincias'].isin(provincias_norte)]

### Almacenamiento y profiling

- Guardamos el datafram combined_df en la carpeta de datos gold como data_card_4_df.csv

In [ ]:
with open(GOLD_DATA_PATH + "data_card_4_df.csv", "w") as f:
    combined_df.to_csv(f, sep=';', encoding='utf-8')

In [ ]:
from  ydata_profiling import ProfileReport

profile = ProfileReport(combined_df, title="Data Card 4 Profiling Report")
profile.to_file(PROFILING_PATH + "data_card_4_profile_report.html")